# Modelos para entrenar

#### Importar librerías

In [100]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np


#### División del Dataset

In [101]:
#path_toia = '/Users/victoria/Desktop/alquiler_procesado.csv'


path_xtrain = '/Users/benjavitale/Documents/ML/TP_F/alquiler_procesado_Xtrain.csv'
path_xtest = '/Users/benjavitale/Documents/ML/TP_F/alquiler_procesado_Xtest.csv'

path_ytrain = '/Users/benjavitale/Documents/ML/TP_F/alquiler_procesado_ytrain.csv'
path_ytest = '/Users/benjavitale/Documents/ML/TP_F/alquiler_procesado_ytest.csv'
X_train = pd.read_csv(path_xtrain, low_memory=False)
X_test = pd.read_csv(path_xtest, low_memory=False)
y_train = pd.read_csv(path_ytrain, low_memory=False)
y_test = pd.read_csv(path_ytest, low_memory=False)


y_train = y_train.values.ravel()
y_test = y_test.values.ravel()


In [102]:
"""path_benja = '/Users/benjavitale/Documents/ML/TP_F/alquiler_procesado.csv'
df = pd.read_csv(path_benja, low_memory=False)"""

"path_benja = '/Users/benjavitale/Documents/ML/TP_F/alquiler_procesado.csv'\ndf = pd.read_csv(path_benja, low_memory=False)"

In [103]:
"""X = df.drop(columns=['precio_pesos_constantes'])  
y = df['precio_pesos_constantes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)"""

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")



X_train shape: (22100, 68)
X_test shape: (5517, 68)
y_train shape: (22100,)
y_test shape: (5517,)


#### Hago función para imprimir métricas

In [104]:
def print_metrics(y_train, y_pred_train, y_test, y_pred_test):
    # Calcular métricas
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)
    
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)
    
    rmse_train = np.sqrt(mse_train)
    rmse_test = np.sqrt(mse_test)

    # Crear DataFrame con las métricas
    metrics = {
        'MAE': [mae_train, mae_test],
        'R²': [r2_train, r2_test],
        'MSE': [mse_train, mse_test],
        'RMSE': [rmse_train, rmse_test]
    }
    metrics_df = pd.DataFrame(metrics, index=['Entrenamiento', 'Prueba'])

    print(metrics_df)


## Regresión Lineal

In [105]:




linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

y_pred_train = linear_model.predict(X_train)
y_pred_test = linear_model.predict(X_test)



print_metrics(y_train, y_pred_train, y_test, y_pred_test)

                        MAE        R²           MSE           RMSE
Entrenamiento  54294.114341  0.624004  1.251644e+10  111876.897108
Prueba         53658.229879  0.659845  1.236731e+10  111208.404642


## XGBOOST

In [114]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
# Suprimir warnings
warnings.filterwarnings("ignore", category=UserWarning)  # Ignorar warnings generales
warnings.filterwarnings("ignore", category=FutureWarning)

# Crear el modelo base
gbr_model = GradientBoostingRegressor(random_state=42)

# Definir los hiperparámetros a buscar
param_grid = {
    'n_estimators': [80, 100],        # Número de árboles en el ensemble
    'learning_rate': [0.5, 0.6],    # Tasa de aprendizaje
    'max_depth': [5, 6],
    'min_samples_split': [2,3],
    'min_samples_leaf': [1,2]               # Máxima profundidad de los árboles         
}

# Configurar GridSearchCV
grid_search = GridSearchCV(
    estimator=gbr_model, 
    param_grid=param_grid, 
    scoring='neg_mean_squared_error',  # Métrica para evaluar
    cv=3,                             # Número de folds para validación cruzada
    verbose=2,                        # Ver progreso
    n_jobs=-1                         # Paralelización
)

# Ejecutar la búsqueda de hiperparámetros
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores Hiperparámetros Encontrados:")
print(best_params)

# Entrenar el modelo con los mejores hiperparámetros
best_gbr_model = grid_search.best_estimator_

# Evaluar el modelo en el conjunto de prueba
y_pred_test = best_gbr_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"MSE en conjunto de prueba: {mse_test:.4f}")
print(f"MAE en conjunto de prueba: {mae_test:.4f}")


Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] END learning_rate=0.5, max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=80; total time=   4.8s
[CV] END learning_rate=0.5, max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=80; total time=   4.9s
[CV] END learning_rate=0.5, max_depth=5, min_samples_leaf=1, min_samples_split=3, n_estimators=80; total time=   4.9s
[CV] END learning_rate=0.5, max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=80; total time=   5.1s
[CV] END learning_rate=0.5, max_depth=5, min_samples_leaf=1, min_samples_split=3, n_estimators=80; total time=   4.8s
[CV] END learning_rate=0.5, max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   6.0s
[CV] END learning_rate=0.5, max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   6.2s
[CV] END learning_rate=0.5, max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=

In [115]:
y_pred_train = best_gbr_model.predict(X_train)
print_metrics(y_train, y_pred_train, y_test, y_pred_test)

                        MAE        R²           MSE          RMSE
Entrenamiento  13927.429719  0.985859  4.707220e+08  21696.129137
Prueba         19310.329274  0.939168  2.211727e+09  47029.004466


In [107]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo Gradient Boosting
gbr_model = GradientBoostingRegressor(
    n_estimators=100, 
    learning_rate=0.6, 
    max_depth=7, 
    random_state=42
)

# Entrenar el modelo
gbr_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_train = gbr_model.predict(X_train)
y_pred_test = gbr_model.predict(X_test)




print_metrics(y_train, y_pred_train, y_test, y_pred_test)

                        MAE        R²           MSE          RMSE
Entrenamiento   7725.735946  0.993649  2.114227e+08  14540.381264
Prueba         16022.650226  0.923034  2.798322e+09  52899.164718


## Red Neuronal

In [108]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Normalizar los datos
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear el modelo
nn_model = Sequential()
nn_model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.001)))
nn_model.add(Dropout(0.2))
nn_model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
nn_model.add(Dropout(0.2))
nn_model.add(Dense(1, activation='linear'))

# Compilar el modelo con tasa de aprendizaje ajustada
optimizer = Adam(learning_rate=0.0005)
nn_model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Entrenar el modelo
history = nn_model.fit(X_train_scaled, y_train, epochs=30, batch_size=16, validation_data=(X_test_scaled, y_test))

# Predicciones
y_pred_train = nn_model.predict(X_train_scaled).ravel()
y_pred_test = nn_model.predict(X_test_scaled).ravel()

# Métricas
print_metrics(y_train, y_pred_train.ravel(), y_test, y_pred_test.ravel())


Epoch 1/30
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 1s 578us/step - loss: 52004253696.0000 - mae: 140224.5938 - val_loss: 42578427904.0000 - val_mae: 82611.3125
Epoch 2/30
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 1s 575us/step - loss: 39477403648.0000 - mae: 79375.9453 - val_loss: 37659680768.0000 - val_mae: 80015.3438
Epoch 3/30
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 1s 522us/step - loss: 33353975808.0000 - mae: 79202.3594 - val_loss: 37135163392.0000 - val_mae: 81067.2656
Epoch 4/30
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 1s 471us/step - loss: 33221328896.0000 - mae: 79311.8281 - val_loss: 36653666304.0000 - val_mae: 81082.7188
Epoch 5/30
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 1s 462us/step - loss: 31446482944.0000 - mae: 78449.4531 - val_loss: 36167553024.0000 - val_mae: 80988.8359
Epoch 6/30
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 1s 469us/step - loss: 31572084736.0000 - mae: 78667.0625 - val_loss: 35652341760.0000 - val_mae: 80951.0469
Epoch 7/30
1382/1382 ━━━━━━━━━━━━━━━━━━━━ 1s 453us/step - loss: 33266561024.0000 - mae: 80969.8516 